In [47]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, GenerationConfig, TrainingArguments, Trainer
from datasets import load_dataset
from torch.optim import AdamW
import numpy as np

In [48]:
huggingface_dataset_name = "knkarthick/dialogsum"
dataset = load_dataset(huggingface_dataset_name)
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 12460
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 500
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary', 'topic'],
        num_rows: 1500
    })
})

In [49]:
tokenizer(dataset['train']['dialogue'][0], return_tensors='pt')

{'input_ids': tensor([[ 1713,   345, 13515,   536,  4663,    10,  2018,     6,  1363,     5,
          3931,     5,    27,    31,    51,  7582, 12833,    77,     7,     5,
          1615,    33,    25,   270,   469,    58,  1713,   345, 13515,   357,
          4663,    10,    27,   435,    34,   133,    36,     3,     9,   207,
           800,    12,   129,     3,     9,   691,    18,   413,     5,  1713,
           345, 13515,   536,  4663,    10,  2163,     6,   168,     6,    25,
            43,    29,    31,    17,   141,    80,    21,   305,   203,     5,
           148,   225,    43,    80,   334,   215,     5,  1713,   345, 13515,
           357,  4663,    10,    27,   214,     5,    27,  2320,    38,   307,
            38,   132,    19,  1327,  1786,     6,   572,   281,   217,     8,
          2472,    58,  1713,   345, 13515,   536,  4663,    10,  1548,     6,
             8,   200,   194,    12,  1792,  2261, 21154,    19,    12,   253,
            91,    81,   135,   778,  

In [50]:
model_name='google/flan-t5-base'

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, torch_dtype=torch.bfloat16)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [51]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,dataset,split):
        self.labels=[tokenizer(lab_prompt,return_tensors='pt') for lab_prompt in dataset[split]['summary']]
        self.text=[tokenizer(text_prompt,return_tensors='pt') for text_prompt in dataset[split]['dialogue']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return self.labels[idx]

    def get_batch_texts(self, idx):
        return self.text[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [61]:
a=Dataset(dataset,'train')
a.get_batch_labels(1)['attention_mask'].squeeze(0)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1])

In [53]:

# Split the dataset into training and validation sets
train_dataset = Dataset(dataset,'train')

# Hyperparameters and Training Configuration
learning_rate = 2e-5
num_epochs = 3

# Define the optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

In [122]:
from tqdm import tqdm

# Fine-tuning function using PEFT
def fine_tune(model, train_dataset, optimizer, num_epochs, batch_size):
    
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(num_epochs):


        for train_input, train_label in tqdm(train_dataloader):
            
            train_label_mask = train_label['attention_mask'].squeeze(1).to(device)
            train_label = train_label['input_ids'].squeeze(1).to(device) # to cuda GPU
            
            mask = train_input['attention_mask'].squeeze(1).to(device) # attention mask
            input_id = train_input['input_ids'].squeeze(1).to(device)

            optimizer.zero_grad()
            outputs = model(input_ids=input_id, attention_mask=mask,decoder_input_ids=train_label,)
            loss = outputs.loss
            print(outputs)
            print(loss)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch: {epoch+1}/{num_epochs}, Average Loss: {total_loss / len(train_dataset)}")

# Fine-tune the model using PEFT
fine_tune(model, train_dataset, optimizer, num_epochs, batch_size)

  0%|                                                                                        | 0/12460 [00:00<?, ?it/s]


ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds

In [114]:
outputs

NameError: name 'outputs' is not defined

In [91]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):